<a href="https://colab.research.google.com/github/Beatrizdacruz/Spark_examples/blob/main/examples/estudoSpark_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark==3.0.1 py4j==0.10.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 24.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612224 sha256=fe34adaaca1bd5369010ffdc3098031c228af124c7ab049a1d9a79ca2bd74759
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:

from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Estudo_PySpark')\
        .getOrCreate()
csv_file = 'data/Top_1000_Companies_Dataset.csv'
#df = spark.read.csv(csv_file)
df = spark.read.format('csv').options(header='true', inferSchema='true', delimiter = ";").load(csv_file)

df.printSchema()
df.columns


data_schema = [
               StructField('company_name', StringType(), True),
               StructField('url', StringType(), True),
               StructField('city', StringType(), True),
               StructField('state', StringType(), True),
               StructField('country', StringType(), True),
               StructField('employees', IntegerType(), True),
               StructField('linkedin_url', StringType(), True),
               StructField('founded', IntegerType(), True),
               StructField('Industry', StringType(), True),
               StructField('GrowjoRanking', IntegerType(), True),
               StructField('PreviousRanking', IntegerType(), True),
               StructField('estimated_revenues', IntegerType(), True),
               StructField('job_openings', IntegerType(), True),
               StructField('keywords', StringType(), True),
               StructField('LeadInvestors', StringType(), True),
               StructField('Accelerator', StringType(), True),
               StructField('btype', StringType(), True),
               StructField('valuation', StringType(), True),
               StructField('total_funding', StringType(), True),
               StructField('product_url', StringType(), True),
               StructField('indeed_url', StringType(), True),
               StructField('growth_percentage', StringType(), True),
               StructField('contact_info', StringType(), True),
            ]

final_struc = StructType(fields = data_schema)

data = spark.read.csv('data/Top_1000_Companies_Dataset.csv',sep = ',', header = True, schema = final_struc)

#data.dtypes
#data.describe().show(5) #mostra as estatísticas
data.show(5)

root
 |-- company_name,url,city,state,country,employees,linkedin_url,founded,Industry,GrowjoRanking,Previous Ranking,estimated_revenues,job_openings,keywords,LeadInvestors,Accelerator,btype,valuation,total_funding,product_url,indeed_url,growth_percentage,contact_info: string (nullable = true)

+-------------------+-----------+-------------+-----+-------------+---------+--------------------+-------+---------+-------------+---------------+------------------+------------+--------------------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+-----------------+--------------------+
|       company_name|        url|         city|state|      country|employees|        linkedin_url|founded| Industry|GrowjoRanking|PreviousRanking|estimated_revenues|job_openings|            keywords|       LeadInvestors|Accelerator|btype|  valuation|total_funding|         product_url|          indeed_url|growth_percentage|        contact_info|
+------------

master()– Se você estiver executando no cluster, precisará usar seu nome mestre como um argumento. normalmente, seria yarn ou mesosdepende da configuração do cluster e também é usado local[X]ao executar no modo autônomo. Xdeve ser um valor inteiro e deve ser maior que 0, que representa quantas partições ele deve criar ao usar RDD, DataFrame e Dataset. Idealmente, o valor Xdeve ser o número de núcleos da CPU.
appName()o método é usado para definir o nome do seu aplicativo.
getOrCreate()o método retorna uma SparkSession existente, caso contrário, cria uma nova SparkSession.

renomear coluna url e remover coluna "contact_info"

In [4]:
data = data.withColumnRenamed('url', 'domain')
data = data.drop('contact_info')
data.show()

+--------------------+-----------------+-------------+-----+-------------+---------+--------------------+-------+--------------+-------------+---------------+------------------+------------+--------------------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+-----------------+
|        company_name|           domain|         city|state|      country|employees|        linkedin_url|founded|      Industry|GrowjoRanking|PreviousRanking|estimated_revenues|job_openings|            keywords|       LeadInvestors|Accelerator|btype|  valuation|total_funding|         product_url|          indeed_url|growth_percentage|
+--------------------+-----------------+-------------+-----+-------------+---------+--------------------+-------+--------------+-------------+---------------+------------------+------------+--------------------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+--------

consultar dados:
Selecionando apenas empresas que foram fundadas após 2020.

In [5]:
from pyspark.sql.functions import col, lit

data.filter(col('founded') >= lit('2020')).show()


+---------------+------------------+-------------+-----+--------------------+---------+--------------------+-------+-------------+-------------+---------------+------------------+------------+--------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+-----------------+
|   company_name|            domain|         city|state|             country|employees|        linkedin_url|founded|     Industry|GrowjoRanking|PreviousRanking|estimated_revenues|job_openings|keywords|       LeadInvestors|Accelerator|btype|  valuation|total_funding|         product_url|          indeed_url|growth_percentage|
+---------------+------------------+-------------+-----+--------------------+---------+--------------------+-------+-------------+-------------+---------------+------------------+------------+--------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+-----------------+
|          Drata|  

buscar empresas que tem entre 500 e 600 funcionários.

In [7]:
data.filter(data.employees.between(500, 600)).show()

+------------------+--------------------+----------------+-----+-------------+---------+--------------------+-------+--------------+-------------+---------------+------------------+------------+--------------------+--------------------+-----------+-----+-----------+-------------+--------------------+--------------------+-----------------+
|      company_name|              domain|            city|state|      country|employees|        linkedin_url|founded|      Industry|GrowjoRanking|PreviousRanking|estimated_revenues|job_openings|            keywords|       LeadInvestors|Accelerator|btype|  valuation|total_funding|         product_url|          indeed_url|growth_percentage|
+------------------+--------------------+----------------+-----+-------------+---------+--------------------+-------+--------------+-------------+---------------+------------------+------------+--------------------+--------------------+-----------+-----+-----------+-------------+--------------------+-----------------

selecionar compania, indústria, palavras-chave e receita estimada ordenada pela receita (decrescente).

In [21]:
colunas = ['company_name', 'Industry', 'keywords', 'estimated_revenues']
data.select(*colunas).orderBy(col('estimated_revenues').desc()).show()

+--------------------+---------------+--------------------+------------------+
|        company_name|       Industry|            keywords|estimated_revenues|
+--------------------+---------------+--------------------+------------------+
|Alexandria Real E...|    Real Estate|                null|        2010000000|
|Installed Buildin...|   Construction|                null|        1880000000|
|  Realty Income Corp|    Real Estate|                null|        1820000000|
|         W. P. Carey|    Real Estate|                null|        1280000000|
|     PGT Innovations|      Materials|                null|        1080000000|
|   Aircastle Limited|        Finance|                null|         750000000|
|AdventHealth West...|Hospital/Health|                null|         750000000|
|National Retail P...|    Real Estate|                null|         702430000|
|     GenusPlus Group|   Construction|                null|         616714000|
|     STAG Industrial|    Real Estate| Finance,Real 